In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import inventorize2 as inv

In [8]:
retail = pd.read_csv('online_retail2.csv')
retail = retail.drop_duplicates()
retail = retail.dropna()
retail = retail[retail.Quantity > 0]

In [11]:
retail_clean = retail.copy()
retail_clean['Revenue'] = retail['Price'] * retail['Quantity']

In [12]:
retail_clean.to_csv('retail_clean.csv')
retail_clean.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country', 'Revenue'],
      dtype='object')

In [15]:
grouped= retail_clean.groupby('Description').agg(total_sales=('Quantity',np.sum),
                                                             total_revenue=('Revenue',np.sum)).reset_index()

In [16]:
grouped.to_csv('for_abc.csv')